#Importing package

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Baseline_MUSS_corrupt/muss
!pip install -e . 
# !pip install -U spacy ==2.0.0
!python -m spacy download en_core_web_md
Ω

Thu Sep  8 22:02:55 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:

from muss.mining.training import get_bart_kwargs, get_score_rows
from muss.resources.prepare import prepare_wikilarge_detokenized, prepare_asset
from muss.resources.datasets import create_smaller_dataset
from muss.fairseq.main import fairseq_train_and_evaluate_with_parametrization
from muss.mining.training import get_score_rows #get_bart_kwargs

#########################
import re
import shutil
import os 

from cachier import cachier
from easse.cli import evaluate_system_output
from easse.utils.constants import TEST_SETS_PATHS
import torch
from tqdm import tqdm

from muss.resources.paths import get_data_filepath, MODELS_DIR, get_dataset_dir
from muss.utils.helpers import add_dicts, args_str_to_dict
from muss.utils.resources import download_and_extract
from muss.preprocessors import GPT2BPEPreprocessor
from muss.preprocessing import apply_line_function_to_file
from muss.fairseq.main import get_language_from_dataset
from muss.text import truncate

from muss.simplify import simplify_sentences
from muss.utils.helpers import *

from muss.simplifiers import get_fairseq_simplifier, get_preprocessed_simplifier
from muss.preprocessors import get_preprocessors, get_preprocessor_by_name

In [ ]:

# This dataset should exist in resources/datasets/ and contain the following files:
# train.complex, train.simple, valid.complex, valid.simple, test.complex, test.simple
def get_evaluate_kwargs(language, phase='valid'):
    return {
        ('en', 'valid'): {'test_set': 'asset_valid'},
        ('en', 'test'): {'test_set': 'asset_test'},
        ('fr', 'valid'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('alector', 'valid', 'complex'),
            'refs_sents_paths': [get_data_filepath('alector', 'valid', 'simple')],
        },
        ('fr', 'test'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('alector', 'test', 'complex'),
            'refs_sents_paths': [get_data_filepath('alector', 'test', 'simple')],
        },
        ('es', 'valid'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('simplext_corpus', 'valid', 'complex'),
            'refs_sents_paths': [get_data_filepath('simplext_corpus', 'valid', 'simple')],
        },
        ('es', 'test'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('simplext_corpus', 'test', 'complex'),
            'refs_sents_paths': [get_data_filepath('simplext_corpus', 'test', 'simple')],
        },
    }[(language, phase)]

evaluate_kwargs=get_evaluate_kwargs('en')


def get_predict_files(language):
    return {
        'en': [get_data_filepath('asset', 'valid', 'complex'), get_data_filepath('asset', 'test', 'complex')],
        'fr': [get_data_filepath('alector', 'valid', 'complex'), get_data_filepath('alector', 'test', 'complex')],
        'es': [
            get_data_filepath('simplext_corpus', 'valid', 'complex'),
            get_data_filepath('simplext_corpus', 'test', 'complex'),
        ],
    }[language]

def get_access_preprocessors_kwargs(language, use_short_name=False):
    return {
        'LengthRatioPreprocessor': {'target_ratio': 0.8, 'use_short_name': use_short_name},
        'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.8, 'use_short_name': use_short_name},
        'WordRankRatioPreprocessor': {'target_ratio': 0.8, 'language': language, 'use_short_name': use_short_name},
        'DependencyTreeDepthRatioPreprocessor': {
            'target_ratio': 0.8,
            'language': language,
            'use_short_name': use_short_name,
        },
    }

def get_bart_kwargs(dataset, language, use_access, use_short_name=False, bart_model='bart.large'):
    assert language == 'en'

    bart_path = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/models/muss_en_wikilarge_mined/model.pt'   ### Change here
    # bart_path = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/models/bart_mined/model.pt'
# 
    arch = {
        'bart.base': 'bart_base',
        'bart.large': 'bart_large',
        'bart.large.cnn': 'bart_large',
    }[bart_model]
    kwargs = {
        'dataset': dataset,
        'metrics_coefs': [0, 1, 0],
        'parametrization_budget': 128,
        'predict_files': get_predict_files(language), # used in fairseq_evaluate_and_save
        'preprocessors_kwargs': {
            'GPT2BPEPreprocessor': {},
        },
        'preprocess_kwargs': {'dict_path': GPT2BPEPreprocessor().dict_path},
        'train_kwargs': {
            'ngpus': 1,
            'arch': arch,
            'restore_file': bart_path,
            'max_tokens': 4096,
            'lr': 8e-05,
            'warmup_updates': 500, #500
            'truncate_source': True,
            'layernorm_embedding': True,
            'share_all_embeddings': True,
            'share_decoder_input_output_embed': True,
            'reset_optimizer': True,
            'reset_dataloader': True,
            'reset_meters': True,
            'required_batch_size_multiple': 1,
            'criterion': 'label_smoothed_cross_entropy',
            'label_smoothing': 0.1,
            'dropout': 0.1,
            'attention_dropout': 0.1,
            'weight_decay': 0.01,
            'optimizer': 'adam',
            'adam_betas': '(0.9, 0.999)',
            'adam_eps': 1e-08,
            'clip_norm': 0.1,
            'lr_scheduler': 'polynomial_decay',
            'max_update': 10000, # 20000
            'skip_invalid_size_inputs_valid_test': True,
            'find_unused_parameters': True,
        },
        'evaluate_kwargs': get_evaluate_kwargs(language),
    }
    if use_access:
        kwargs['preprocessors_kwargs'] = add_dicts(
            get_access_preprocessors_kwargs(language, use_short_name=use_short_name), kwargs['preprocessors_kwargs']
        )
    return kwargs

### Compute hardword word retention rate 
def find_occurence(NE,simple_sentence):
  NE_simp = [i.lower() for i in NE['NE_simple']]
  NE_diff = [i.lower() for i in NE['NE_difficult']]
  simple_sentence = simple_sentence.lower()
  intersect_words = []

  count_simple = 0
  count_difficult = 0
  # print(NE_diff)
  # print(NE)
  if NE_simp[0] != '' or NE_diff:
    # print(NE_simp[0] == '')
  # print(sentence,test_complex_NER_spacy[index])
    if NE_simp[0] != '':
      for text in NE_simp:
        if text.lower() in simple_sentence:
          intersect_words.append(text)
          count_simple +=1
    else:
      count_simple = None
    
    if NE_diff:
      for text in NE_diff:
        if text.lower() in simple_sentence:
          intersect_words.append(text)
          count_difficult +=1
          # print('diff words found in sen')
    else:
      count_difficult = None

    return count_simple, count_difficult
  else:
    # print('empty NE')
    return None, None
    
    
def word_retention_rate(comp_sentence,simp_sentence):
  total_simp = 0
  total_diff = 0
  counter_simp = 0
  counter_diff = 0

  for i,sentence in enumerate(simp_sentence):
    NE_temp1 = NE_extraction(comp_sentence[i])[0]

    count_simp, count_diff = find_occurence(NE_temp1,sentence)

    if count_simp != None or count_diff != None:
      if count_simp != None:
        total_simp += len(NE_temp1['NE_simple'])
        counter_simp += count_simp
      if count_diff != None:
        total_diff += len(NE_temp1['NE_difficult'])
        counter_diff += count_diff
    else:
      pass
    # print(count_simp, count_diff)
  if total_diff != 0 and total_simp!=0:
    print('-'*50)
    print('simple_NE retention rate',counter_simp/total_simp)
    print('difficult_NE retention rate',counter_diff/total_diff)
    print(total_diff)
    print('total_NE retention rate', (counter_simp+counter_diff)/(total_simp+total_diff))
    print('-'*50)
    return counter_simp/total_simp,counter_diff/total_diff, (counter_simp+counter_diff)/(total_simp+total_diff)

  else:
    if total_simp != 0:
      print('-'*50)
      print('simple_NE retention rate',counter_simp/total_simp)
      print('-'*50)
      return counter_simp/total_simp
    if total_diff != 0:
      print('-'*50)
      print('difficult_NE retention rate',counter_diff/total_diff)
      # print('total_NE retention rate', (counter_simp+counter_diff)/(total_simp+total_diff))
      print('-'*50)
      # print('No stats provided for diff_NE becuase no diff NE at all')
    



def word_retention_rate_per_sentence(comp_sentence,simp_sentence):
  total_simp_list = []
  total_diff_list = []
  total_list = []
  # counter_simp_list = []
  # counter_diff_list = []

  for i,sentence in enumerate(simp_sentence):
    total_simp = 0
    total_diff = 0
    counter_simp = 0
    counter_diff = 0

    NE_temp1 = NE_extraction(comp_sentence[i])[0]

    count_simp, count_diff = find_occurence(NE_temp1,sentence)
    if count_simp != None or count_diff != None:
      if count_simp != None:
        total_simp += len(NE_temp1['NE_simple'])
        counter_simp += count_simp
      if count_diff != None:
        total_diff += len(NE_temp1['NE_difficult'])
        counter_diff += count_diff
    else:
      pass
    # print(count_simp, count_diff)
    if total_diff != 0:
      # print('simple_NE retention rate',counter_simp/total_simp)
      # print('difficult_NE retention rate',counter_diff/total_diff)
      # print('total_NE retention rate', (counter_simp+counter_diff)/(total_simp+total_diff))
      total_diff_list.append(counter_diff/total_diff)
    if total_simp != 0:
      total_simp_list.append(counter_simp/total_simp)
    if total_simp != 0 or total_diff != 0:
      total_list.append((counter_simp+counter_diff)/(total_simp+total_diff))
      # if type(counter_diff/total_diff) == float:
      #   total_diff_list.append(counter_diff/total_diff)
      # else:
      #   print(counter_diff,total_diff)
      #   total_diff_list.append(counter_diff/total_diff)

      # return counter_simp/total_simp,counter_diff/total_diff, (counter_simp+counter_diff)/(total_simp+total_diff)
    # else:
    #   total_simp_list.append(counter_simp/total_simp)
    #   total_list.append((counter_simp+counter_diff)/(total_simp+total_diff))
  print('-'*50)
  print('Word retention rate per sentence')
  print('simple_NE retention rate',np.average(total_simp_list))
  print('difficult_NE retention rate',np.average(total_diff_list))
  # print(total_diff_list)
  print('total_NE retention rate',np.average(total_list))
  print('-'*50)


def generate_output(chosen_id_list,data_dir_list,data_list,model_dir_dict,sample=False,add_to_dir_name=None,muss_output=False,NE_output=False,CERF_output=False,NE_CERF_output=False,hypothesis_num=None,beam=None,sampling=False):

  # store created dir
  created_dir = []
  output_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/'
  for index in chosen_id_list:
    
    # get model unique local id
    exp_dir = model_dir_dict[index]['exp_dir']
    print('| model_name is',model_dir_dict[index]['model_name'])
    print('| exp_dir is',exp_dir)

    model_name = 'model_'+ str(index) + '_'+ exp_dir.split('/')[-2]

    os.makedirs(output_dir+model_name, exist_ok=True)
    print('| output_dir',output_dir)
    language = 'en'

    # get recommended_preprocessors_kwargs
    if 'recommended_preprocessors_kwargs' in model_dir_dict[index].keys() :
      preprocessors_kwargs = model_dir_dict[index]['recommended_preprocessors_kwargs']
      print('| use provided kwargs')
    else:
      # use muss_mined_wiki preprocessors
      preprocessors_kwargs = {
            'LengthRatioPreprocessor': {'target_ratio': 0.9, 'use_short_name': False},
            'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.65, 'use_short_name': False},
            'WordRankRatioPreprocessor': {'target_ratio': 0.75, 'language': language, 'use_short_name': False},
            'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 0.4, 'language': language, 'use_short_name': False},
        }
      preprocessors_kwargs['GPT2BPEPreprocessor'] = {}

    print('| use preprocessors_kwargs',preprocessors_kwargs)
    preprocessors = get_preprocessors(preprocessors_kwargs)

    if hypothesis_num == None:
      hypothesis_num = 1
    if beam == None:
      beam=5

    generate_kwargs = {'hypothesis_num':hypothesis_num,
                       'beam':beam,
                       'sampling':sampling}

    simplifier = get_fairseq_simplifier(exp_dir, **generate_kwargs)
    simplifier = get_preprocessed_simplifier(simplifier, preprocessors=preprocessors)

    #########################################
    # simplify sample sentences 

    ############################## I need to change the directory 
    if sample:
      if muss_output:
        complex_file_dir = '/content/drive/MyDrive/muss/scripts/contract_no_token.en'
      elif NE_output:
        complex_file_dir = '/content/drive/MyDrive/muss/scripts/contract_NE_token.en'
      elif CERF_output:
        complex_file_dir = '/content/drive/MyDrive/muss/scripts/contract_ABCD_token.en'
      elif NE_CERF_output:
        complex_file_dir = '/content/drive/MyDrive/muss/scripts/contract_NE_ABCD_token.en'

      pred_path = simplifier(complex_file_dir)

      for i in range(len(read_lines(complex_file_dir))):
        print('original:\n',read_lines(complex_file_dir)[i])
        print('simplified:\n',read_lines(pred_path)[i])
        print('----------------------------------------------------')

      return read_lines(complex_file_dir),read_lines(pred_path)
    ###########################################

    # create dir for output
    subfolders = os.listdir(output_dir + model_name)
    num_subfolder = len(subfolders)
    new_folder_dir = output_dir + model_name + '/' +str(num_subfolder).zfill(2) +'/'
    os.mkdir(new_folder_dir)
    created_dir.append(new_folder_dir)

     # simplify asset dataset
    for index, file_dir in enumerate(data_dir_list):
      
      info_file_name = new_folder_dir + 'info.txt'
      with open(info_file_name, 'w') as f:
        f.write("generate_kwargs %s\n" % generate_kwargs)
      f.close()

      filename = new_folder_dir + data_list[index]
      print('filename;', filename)
      pred_path = simplifier(file_dir)

      with open(filename, 'w') as f:
          for item in read_lines(pred_path):
              f.write("%s\n" % item)

  print('created_dir',created_dir)
  return created_dir


def word_retention_rate_NER(NER,simp_sentence):
  total_simp = 0
  total_diff = 0
  counter_simp = 0
  counter_diff = 0

  for i,sentence in enumerate(simp_sentence):
    NE_temp1 = {'NE_simple':NER[i],'NE_difficult':[]}

    count_simp, count_diff = find_occurence(NE_temp1,sentence)

    if count_simp != None or count_diff != None:
      if count_simp != None:
        total_simp += len(NE_temp1['NE_simple'])
        counter_simp += count_simp
      if count_diff != None:
        total_diff += len(NE_temp1['NE_difficult'])
        counter_diff += count_diff
    else:
      pass
    # print(count_simp, count_diff)
  if total_diff != 0 and total_simp!=0:
    print('-'*50)
    print('simple_NE retention rate',counter_simp/total_simp)
    print('difficult_NE retention rate',counter_diff/total_diff)
    print(total_diff)
    print('total_NE retention rate', (counter_simp+counter_diff)/(total_simp+total_diff))
    print('-'*50)
    return counter_simp/total_simp,counter_diff/total_diff, (counter_simp+counter_diff)/(total_simp+total_diff)

  else:
    if total_simp != 0:
      print('-'*50)
      print('simple_NE retention rate',counter_simp/total_simp)
      print('-'*50)
      return counter_simp/total_simp
    if total_diff != 0:
      print('-'*50)
      print('difficult_NE retention rate',counter_diff/total_diff)
      # print('total_NE retention rate', (counter_simp+counter_diff)/(total_simp+total_diff))
      print('-'*50)
      # print('No stats provided for diff_NE becuase no diff NE at all')


def reconstruct_simplified_data_extra_NE(complex_sentence, simple_sentence):        ### reconstruct the sentence with only NE that doesn't appear

    control_tokens = [" <NEXT_NE> "," <NEXT_DIFFICULT_WORD> "," <SENT_START> "]
    # print('-'*10)
    temp_NE = NE_extraction(complex_sentence)[0]
    new_NE = {'NE_simple':[],'NE_difficult':[]}
    # print(temp_NE)
    for key, value in temp_NE.items():
      for word in temp_NE[key]:
        if word != '' and word != ['']:
          if word not in simple_sentence:
            new_NE[key].append(word)
        else:
          # print(word)
          pass
    # print(new_NE,'---')

#################
    # indice_of_start = list_duplicates_of(complex_sentence,control_tokens[2])
    sentence = control_tokens[0] + control_tokens[0].join(new_NE['NE_simple']) + control_tokens[2] + simple_sentence
#################
    # Change here to eliminate difficult words 
    # indice_of_start_diff = list_duplicates_of(complex_sentence,control_tokens[1])

    # sentence = control_tokens[0] + control_tokens[0].join(new_NE['NE_simple']) + control_tokens[1].join(new_NE['NE_difficult']) + control_tokens[2] + simple_sentence
#################    
    return sentence

def reconstruct_simplified_data(complex_sentence, simple_sentence):
    
    def list_duplicates_of(seq,item):
      start_at = -1
      locs = []
      while True:
          try:
              loc = seq.index(item,start_at+1)
          except ValueError:
              break
          else:
              locs.append(loc)
              start_at = loc
      return locs
    
    control_tokens = ["<NEXT_NE> ","<NEXT_DIFFICULT_WORD>","<SENT_START>"]


#################
    indice_of_start = list_duplicates_of(complex_sentence,control_tokens[2])
    sentence = complex_sentence[0:indice_of_start[0]] + control_tokens[2] + simple_sentence
#################
    # Change here to eliminate difficult words 
    # indice_of_start_diff = list_duplicates_of(complex_sentence,control_tokens[1])
    # sentence = complex_sentence[0:indice_of_start_diff[0]] + control_tokens[2] + simple_sentence
#################    
    return sentence


#loading datasets

In [ ]:
asset_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/'
asset_ne_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset_ne2/'

asset_valid = asset_dir + 'valid.complex'
asset_test = asset_dir + 'test.complex'

asset_ne_valid = asset_ne_dir + 'valid.complex'
asset_ne_test = asset_ne_dir + 'test.complex'

asset_valid_NER = [NE_extraction(i)[0]['NE_simple'] for i in read_lines(asset_ne_valid)]
asset_test_NER = [NE_extraction(i)[0]['NE_simple'] for i in read_lines(asset_ne_test)]

asset_valid_NER = [list(set(i)) for i in asset_valid_NER]
asset_test_NER = [list(set(i)) for i in asset_test_NER]

Temp_data_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/qualitative/'
Temp_data_name = 'process_sentence1'

# NE_retention_prefix all 

## Simplify asset using baseline MUSS

In [ ]:
simp_sentence_baseline = simplify_sentences(read_lines(asset_test))
word_retention_rate_NER(asset_test_NER,simp_sentence_baseline)
baseline_path = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/muss_wikilarge_mined/00/'
# simp_asset_test_baseline = simplify_sentences(read_lines(asset_test_dir))

with open(baseline_path+'asset_test_baseline','w') as f:
  for sen in simp_sentence_baseline:
    f.write("{}\n".format(sen))
baseline_test = baseline_path + 'asset_test_baseline' 
ast.literal_eval(os.popen("easse evaluate -t asset_test -m 'bleu,sari,fkgl' -q < %s" %baseline_test).read())

/usr/local/lib/python3.7/dist-packages/fairseq/search.py:140: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  beams_buf = indices_buf // vocab_size
/usr/local/lib/python3.7/dist-packages/fairseq/sequence_generator.py:651: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  unfin_idx = idx // beam_size


--------------------------------------------------
simple_NE retention rate 0.5947521865889213
--------------------------------------------------


{'bleu': 55.998,
 'sari': 43.547,
 'fkgl': 5.46,
 'quality_estimation': {'Compression ratio': 0.898,
  'Sentence splits': 1.451,
  'Levenshtein similarity': 0.7,
  'Exact copies': 0.0,
  'Additions proportion': 0.311,
  'Deletions proportion': 0.334,
  'Lexical complexity score': 7.762}}

In [ ]:
### Reconstruct the data
sentences_with_prefix = []
for i,sentence in enumerate(simp_sentence_baseline):
  sentence_with_control_token = " <NEXT_NE> " + " <NEXT_NE> ".join(asset_test_NER[i]) + " <SENT_START> " + sentence
  sentences_with_prefix.append(sentence_with_control_token)
sentences_with_prefix[:10]

### Find the sentence that dont have any prefix existed

comp_sentence1_interstage = []
sentence_no_retention_index = [] ###
for i, sen in enumerate(sentences_with_prefix):
  sentence = reconstruct_simplified_data_extra_NE(sen,simp_sentence_baseline[i])
  tempNE = NE_extraction(sentence)[0]

  var = 1
  tempNE_list = []
  for key,value in tempNE.items():
    # print(value)
    for k in value:
      tempNE_list.append(k)
  # print(tempNE_list)
  if tempNE_list[0] != '':
    var = 0   ### Since the NE list is not None 

  if var == 1:
    comp_sentence1_interstage.append(sen)
  else:
    comp_sentence1_interstage.append(sentence)
    sentence_no_retention_index.append(i)

  # print('-----')

# print(comp_sentence1_interstage[:10])
# len(sentence_no_retention_index)

zeropercent_retention_sentence = [comp_sentence1_interstage[i] for i in sentence_no_retention_index]
print(zeropercent_retention_sentence[0:10])
word_retention_rate(zeropercent_retention_sentence,[NE_extraction(i)[1] for i in zeropercent_retention_sentence]) 
print(len(zeropercent_retention_sentence))
print('NO NE has been RETAINED')

with open(Temp_data_dir+Temp_data_name,'w') as f:
  # print(type(sentence))
  for sen in zeropercent_retention_sentence:
    f.write("{}\n".format(sen))
f.close()


[' <NEXT_NE> Rizeigat <NEXT_NE> Janjaweed, <SENT_START> One side of the armed conflicts is composed mainly of the Sudanese military and the Janjaweed. The Janjaweed are a militia group made up mostly of Abbala tribes from the northern part of Sudan.', " <NEXT_NE> Islam's <SENT_START> Jeddah is the main city on the road to Mecca. Mecca is the Islamic world's holiest city, and Muslims must visit it at least once a year.", ' <NEXT_NE> thought <SENT_START> The Great Dark Spot is the biggest dark spot on Neptune. It looks like a hole.', ' <NEXT_NE> neurosurgeon. <SENT_START> In his next work, Saturday, a successful neurosurgeon has an especially eventful day.', ' <NEXT_NE> away <NEXT_NE> attaching <NEXT_NE> and, <NEXT_NE> tarantula, <NEXT_NE> character, <NEXT_NE> ball, <SENT_START> The trickster character then spun a black cord and attached it to the ball. He then crawled fast to the east and pulled on the cord with all his strength.', ' <NEXT_NE> culturally <NEXT_NE> akin <SENT_START> Thei

## Test the recovery effect- using zero-percent retention rate sentence

> Indented block



In [ ]:
MODEL_DIR = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/'

asset_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset/'
asset_ne_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset_ne2/'
asset_tag = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset_tag/'
asset_prefix_tag = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/asset_prefix_tag/'

NE_recovery_prefix = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/NE_recovery_prefix_all/'


model_dir_dict = {}
def add_item_to_dict(**kwargs):

  id = len(model_dir_dict)
  model_dir_dict[id]={}

  for key, value in kwargs.items():
    model_dir_dict[id][key]=value
  print('added:',model_dir_dict[id])

add_item_to_dict(model_id=len(model_dir_dict),exp_dir= MODEL_DIR+'local_1662418124367/',model_name='NE_retention_prefix_all',recommended_preprocessors_kwargs={'LengthRatioPreprocessor': {'target_ratio': 0.7324409180496487, 'use_short_name': False}, 'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.960122996148627, 'use_short_name': False}, 'WordRankRatioPreprocessor': {'target_ratio': 1.026913449646228, 'language': 'en', 'use_short_name': False}, 'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 1.486515239255297, 'language': 'en', 'use_short_name': False}, 'GPT2BPEPreprocessor': {}})
#load test dataset 
test_data_dir_list,test_data_list = [Temp_data_dir+Temp_data_name],['test.complex.pred']
model_dir_dictx = model_dir_dict

model_dir_dictx[0]

added: {'model_id': 0, 'exp_dir': '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/local_1662418175945/', 'model_name': 'NE_retention_prefix_all', 'recommended_preprocessors_kwargs': {'LengthRatioPreprocessor': {'target_ratio': 0.7324409180496487, 'use_short_name': False}, 'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.960122996148627, 'use_short_name': False}, 'WordRankRatioPreprocessor': {'target_ratio': 1.026913449646228, 'language': 'en', 'use_short_name': False}, 'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 1.486515239255297, 'language': 'en', 'use_short_name': False}, 'GPT2BPEPreprocessor': {}}}


{'model_id': 0,
 'exp_dir': '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/local_1662418175945/',
 'model_name': 'NE_retention_prefix_all',
 'recommended_preprocessors_kwargs': {'LengthRatioPreprocessor': {'target_ratio': 0.7324409180496487,
   'use_short_name': False},
  'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.960122996148627,
   'use_short_name': False},
  'WordRankRatioPreprocessor': {'target_ratio': 1.026913449646228,
   'language': 'en',
   'use_short_name': False},
  'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 1.486515239255297,
   'language': 'en',
   'use_short_name': False},
  'GPT2BPEPreprocessor': {}}}

In [ ]:
token_ori_1 = generate_output([0],test_data_dir_list,test_data_list, model_dir_dictx)
comp_sentence1, simp_sentence1 = show_simplify(test_data_dir_list[0],token_ori_1[0]+test_data_list[0])



| model_name is NE_retention_prefix_all
| exp_dir is /content/drive/MyDrive/Baseline_MUSS_corrupt/muss/experiments/fairseq/local_1662418175945/
| output_dir /content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/
| use provided kwargs
| use preprocessors_kwargs {'LengthRatioPreprocessor': {'target_ratio': 0.7324409180496487, 'use_short_name': False}, 'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.960122996148627, 'use_short_name': False}, 'WordRankRatioPreprocessor': {'target_ratio': 1.026913449646228, 'language': 'en', 'use_short_name': False}, 'DependencyTreeDepthRatioPreprocessor': {'target_ratio': 1.486515239255297, 'language': 'en', 'use_short_name': False}, 'GPT2BPEPreprocessor': {}}
filename; /content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/model_0_local_1662418175945/00/test.complex.pred
created_dir ['/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/output/model_0_local_1662418175945/00/']
simplified:

-------------------------------------------------------

In [ ]:
# word_retention_rate_NER(asset_test_NER,simp_sentence1)
word_retention_rate(comp_sentence1,simp_sentence1)

--------------------------------------------------
simple_NE retention rate 0.7132701421800948
--------------------------------------------------


0.7132701421800948

In [ ]:
reconstruct_data = simp_sentence_baseline.copy()
for i,j in enumerate(sentence_no_retention_index):
  # print(j)
  # print(reconstruct_data[j], '\n',simp_sentence1[i])
  reconstruct_data[j] = simp_sentence1[i]
  # print(reconstruct_data[j])
  # print('-')


In [ ]:
word_retention_rate_NER(asset_test_NER,reconstruct_data)


--------------------------------------------------
simple_NE retention rate 0.8658892128279884
--------------------------------------------------


0.8658892128279884

In [ ]:
with open(Temp_data_dir+'reconstruct_data','w') as f:
  # print(type(sentence))
  for sen in reconstruct_data:
    f.write("{}\n".format(sen))
f.close()
ast.literal_eval(os.popen("easse evaluate -t asset_test -m 'bleu,sari,fkgl' -q < %s" %Temp_data_dir+'reconstruct_data').read())

{'bleu': 54.776,
 'sari': 42.705,
 'fkgl': 5.718,
 'quality_estimation': {'Compression ratio': 0.928,
  'Sentence splits': 1.499,
  'Levenshtein similarity': 0.706,
  'Exact copies': 0.006,
  'Additions proportion': 0.29,
  'Deletions proportion': 0.299,
  'Lexical complexity score': 7.897}}

#Training 


In [ ]:

# This dataset should exist in resources/datasets/ and contain the following files:
# train.complex, train.simple, valid.complex, valid.simple, test.complex, test.simple
def get_evaluate_kwargs(language, phase='valid'):
    return {
        ('en', 'valid'): {'test_set': 'asset_valid'},
        ('en', 'test'): {'test_set': 'asset_test'},
        ('fr', 'valid'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('alector', 'valid', 'complex'),
            'refs_sents_paths': [get_data_filepath('alector', 'valid', 'simple')],
        },
        ('fr', 'test'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('alector', 'test', 'complex'),
            'refs_sents_paths': [get_data_filepath('alector', 'test', 'simple')],
        },
        ('es', 'valid'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('simplext_corpus', 'valid', 'complex'),
            'refs_sents_paths': [get_data_filepath('simplext_corpus', 'valid', 'simple')],
        },
        ('es', 'test'): {
            'test_set': 'custom',
            'orig_sents_path': get_data_filepath('simplext_corpus', 'test', 'complex'),
            'refs_sents_paths': [get_data_filepath('simplext_corpus', 'test', 'simple')],
        },
    }[(language, phase)]

evaluate_kwargs=get_evaluate_kwargs('en')


def get_predict_files(language):
    return {
        'en': [get_data_filepath('asset', 'valid', 'complex'), get_data_filepath('asset', 'test', 'complex')],
        'fr': [get_data_filepath('alector', 'valid', 'complex'), get_data_filepath('alector', 'test', 'complex')],
        'es': [
            get_data_filepath('simplext_corpus', 'valid', 'complex'),
            get_data_filepath('simplext_corpus', 'test', 'complex'),
        ],
    }[language]

def get_access_preprocessors_kwargs(language, use_short_name=False):
    return {
        'LengthRatioPreprocessor': {'target_ratio': 0.8, 'use_short_name': use_short_name},
        'ReplaceOnlyLevenshteinPreprocessor': {'target_ratio': 0.8, 'use_short_name': use_short_name},
        'WordRankRatioPreprocessor': {'target_ratio': 0.8, 'language': language, 'use_short_name': use_short_name},
        'DependencyTreeDepthRatioPreprocessor': {
            'target_ratio': 0.8,
            'language': language,
            'use_short_name': use_short_name,
        },
    }

# def get_bart_kwargs(dataset, language, use_access, use_short_name=False, bart_model='bart.large'):
#     assert language == 'en'

#     # bart_path = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/models/muss_en_wikilarge_mined/model.pt' 
#     bart_path = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/models/bart.large/model.pt' 

#     arch = {
#         'bart.base': 'bart_base',
#         'bart.large': 'bart_large',
#         'bart.large.cnn': 'bart_large',
#     }[bart_model]
#     kwargs = {
#         'dataset': dataset,
#         'metrics_coefs': [0, 1, 0],
#         'parametrization_budget': 128,
#         'predict_files': get_predict_files(language), # used in fairseq_evaluate_and_save
#         'preprocessors_kwargs': {
#             'GPT2BPEPreprocessor': {},
#         },
#         'preprocess_kwargs': {'dict_path': GPT2BPEPreprocessor().dict_path},
#         'train_kwargs': {
#             'ngpus': 1,
#             'arch': arch,
#             'restore_file': bart_path,
#             'max_tokens': 4096,
#             'lr': 3e-05,
#             'warmup_updates': 500, #500
#             'truncate_source': True,
#             'layernorm_embedding': True,
#             'share_all_embeddings': True,
#             'share_decoder_input_output_embed': True,
#             'reset_optimizer': True,
#             'reset_dataloader': True,
#             'reset_meters': True,
#             'required_batch_size_multiple': 1,
#             'criterion': 'label_smoothed_cross_entropy',
#             'label_smoothing': 0.1,
#             'dropout': 0.1,
#             'attention_dropout': 0.1,
#             'weight_decay': 0.01,
#             'optimizer': 'adam',
#             'adam_betas': '(0.9, 0.999)',
#             'adam_eps': 1e-08,
#             'clip_norm': 0.1,
#             'lr_scheduler': 'polynomial_decay',
#             'max_update': 10000, # 20000
#             'skip_invalid_size_inputs_valid_test': True,
#             'find_unused_parameters': True,
#         },
#         'evaluate_kwargs': get_evaluate_kwargs(language),
#     }
#     if use_access:
#         kwargs['preprocessors_kwargs'] = add_dicts(
#             get_access_preprocessors_kwargs(language, use_short_name=use_short_name), kwargs['preprocessors_kwargs']
#         )
#     return kwargs



def get_bart_kwargs(dataset, language, use_access, use_short_name=False, bart_model='bart.large'):
    assert language == 'en'
    bart_path = prepare_bart_model(bart_model) / 'model.pt'
    # bart_path = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/models/muss_en_wikilarge_mined/model.pt'
    arch = {
        'bart.base': 'bart_base',
        'bart.large': 'bart_large',
        'bart.large.cnn': 'bart_large',
    }[bart_model]
    kwargs = {
        'dataset': dataset,
        'metrics_coefs': [0, 1, 0],
        'parametrization_budget': 128,
        'predict_files': get_predict_files(language),
        'preprocessors_kwargs': {
            'GPT2BPEPreprocessor': {},
        },
        'preprocess_kwargs': {'dict_path': GPT2BPEPreprocessor().dict_path},
        'train_kwargs': {
            'ngpus': 8,
            'arch': arch,
            'restore_file': bart_path,
            'max_tokens': 4096,
            'lr': 3e-05,
            'warmup_updates': 500,
            'truncate_source': True,
            'layernorm_embedding': True,
            'share_all_embeddings': True,
            'share_decoder_input_output_embed': True,
            'reset_optimizer': True,
            'reset_dataloader': True,
            'reset_meters': True,
            'required_batch_size_multiple': 1,
            'criterion': 'label_smoothed_cross_entropy',
            'label_smoothing': 0.1,
            'dropout': 0.1,
            'attention_dropout': 0.1,
            'weight_decay': 0.01,
            'optimizer': 'adam',
            'adam_betas': '(0.9, 0.999)',
            'adam_eps': 1e-08,
            'clip_norm': 0.1,
            'lr_scheduler': 'polynomial_decay',
            'max_update': 20000,
            'skip_invalid_size_inputs_valid_test': True,
            'find_unused_parameters': True,
        },
        'evaluate_kwargs': get_evaluate_kwargs(language),
    }
    if use_access:
        kwargs['preprocessors_kwargs'] = add_dicts(
            get_access_preprocessors_kwargs(language, use_short_name=use_short_name), kwargs['preprocessors_kwargs']
        )
    return kwargs

In [ ]:
dataset = 'NE_recovery_prefix_3'
data_dir = '/content/drive/MyDrive/Baseline_MUSS_corrupt/muss/resources/datasets/'
read_lines(data_dir+dataset+'/train.complex')[:10]

[' <NEXT_NE> evidence <SENT_START> There is manuscript support that Austen continued to work on these pieces as late as the period 1809 â "11',
 ' <NEXT_NE> analysis <SENT_START> In a remarkable comparative study',
 ' <NEXT_NE> legend <SENT_START> Plays and comic puppet theater loosely based on this story were popular throughout Germany in the 16th century',
 ' <NEXT_NE> electricity <NEXT_NE> Algiers <NEXT_NE> allegedly <SENT_START> The Algerian government cut the power supply to at least large parts of the capital city (and also to large parts of the country)',
 ' <NEXT_NE> environments <NEXT_NE> relies <NEXT_NE> signal <NEXT_NE> transduction. <NEXT_NE> disease <SENT_START> Sensing of both the external and internal world at the cellular level depends on information production Many health processes',
 ' <NEXT_NE> footbridge <SENT_START> A very wide covered bridge joins all platforms at their western ends but does not provide entry to or egress from the station.',
 ' <NEXT_NE> presence 

In [ ]:
# dataset = 'NE_recovery'
kwargs = get_bart_kwargs(dataset=dataset, language='en', use_access=True)
kwargs['train_kwargs']['ngpus'] = 1  # Set this from 8 to 1 for local training
kwargs['train_kwargs']['max_tokens'] = 512  # Lower this number to prevent OOM
kwargs['train_kwargs']['warmup_updates'] = 1000
kwargs['train_kwargs']['max_update'] = 3500
kwargs['train_kwargs']['save_interval_updates'] = 1000
result = fairseq_train_and_evaluate_with_parametrization(**kwargs)
print('Training done')

NameError: ignored

In [ ]:
while True:pass